# Speedtest data exploration

### How data is  collected

Running speedtest application (?) similar to speedtest.net Calculating upload, download speed + png latency

### How data looks like
Load libraries:

In [ ]:
from data_exploration import *

In [ ]:
import pyodbc
with open('/home/connectin/config.json', 'r') as f:
    main_config = json.load(f)
def connect_to_mssql():
    connection = pyodbc.connect(driver=main_config['driver'], server=os.environ['MSSQL_HOST'],
                                port=os.environ['MSSQL_PORT'], uid=os.environ['MSSQL_USER'],
                                pwd=os.environ['MSSQL_PASSWORD'], database=os.environ['MSSQL_DATABASE'])
    return connection

Set up test time interval:

In [ ]:
time_interval='4w' #5d

In [ ]:
#Set up starting point, by default if will start from current time
starting_point=datetime.now().strftime('%Y-%m-%d %H:%M:%S')
#starting point="2019-01-10 14:00:00"  # to set upl alternative starting point
print("Starting point:",starting_point )

Set up influxdb connection:

In [ ]:
client, client_df = connect_to_influxdb()

Checking last 10 records for the SPEEDTEST_UPLOAD  measurment and see how data looks like:

In [ ]:
query_upload = "SELECT * FROM SPEEDTEST_IPERF_UPLOAD WHERE PROVIDER!='iperf' ORDER BY time DESC LIMIT 10;"
upload_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_upload,table_name='SPEEDTEST_IPERF_UPLOAD')
upload_df

Let's take just one device, for example 3:

In [ ]:
query_upload = "SELECT * FROM SPEEDTEST_IPERF_UPLOAD WHERE PROVIDER!='iperf' AND SK_PI='3' ORDER BY time DESC LIMIT 10;"
upload_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_upload,table_name='SPEEDTEST_IPERF_UPLOAD')
upload_df

Let's compare with what we have in MS SQL database:

In [ ]:
cnxn = connect_to_mssql()
sql = "SELECT TOP 10 * FROM FCT_SPEEDTEST WHERE PROVIDER!='iperf' AND SK_PI='3' ORDER BY DATA_DATE DESC;"
pd.read_sql(sql,cnxn)

Are there any zeros in mssql database?

In [ ]:
sql = "SELECT * FROM FCT_SPEEDTEST WHERE PROVIDER!='iperf' AND (UPLOAD=0 OR DOWNLOAD=0 OR PING=0);"
pd.read_sql(sql,cnxn)

Something is possibly wrong with these measurments, we wille exclude zeros for now.

### How may data points per device?

Getting device numbers(tags SK_PI):

In [ ]:
device_numbers=get_tag_values_influxdb(client_influx=client,table_name='SPEEDTEST_IPERF_UPLOAD', tag_name='SK_PI')
device_numbers=list(map(int, device_numbers))
device_numbers= sorted(device_numbers)
print(device_numbers)

Getting number of data points per device for the entire period of time.

In [ ]:
query_upload_counts = "SELECT COUNT(UPLOAD) FROM SPEEDTEST_IPERF_UPLOAD WHERE PROVIDER!='iperf' AND time<= '"+starting_point+"' AND UPLOAD>0 GROUP BY SK_PI;"
upload_counts=get_stats_influxdb(client_influx=client,
                               query_influx=query_upload_counts,
                               stat_name='count',
                               device_numbers=device_numbers)

Plotting device numbers and number of data points. 

In [ ]:
simple_bar_plot(xvalues=device_numbers,
                yvalues=upload_counts,
                name="upload datapoints",
                title="Number of data points per device to the date "+starting_point,
                ytitle="Number of datapoints")

Some of the devices have small number of datapoints, may be they are just installed? Lets check how many dataponts came in last 4 weeks.

Getting number of datapoints per device in last 4 weeks.

In [ ]:
query_upload_counts_time = "SELECT COUNT(UPLOAD) FROM SPEEDTEST_IPERF_UPLOAD WHERE time >= '"+starting_point+"'-"+time_interval+" AND PROVIDER!='iperf' AND UPLOAD>0 GROUP BY SK_PI ;"
upload_counts_time = get_stats_influxdb(client_influx=client,
                                      query_influx=query_upload_counts_time,
                                      stat_name='count',
                                      device_numbers=device_numbers)

Plotting combined barchart - entire number of datapoints vs number of datapoints in last 4 weeks.

In [ ]:
combined_bar_plot_2traces(xvalues=device_numbers,
                          yvalues1=upload_counts_time,
                          yvalues2=[a - b for a, b in zip(upload_counts, upload_counts_time)],
                          name1='Last '+time_interval,
                          name2='The rest of the time',
                          title="Comparing number of datapoints in last "+time_interval+" vs entire time starting from "+starting_point,
                          ytitle="Number of datapoints")

Devices 1,2,4,6 and 8 have data but not in the last 4 weeks.Let's check last reporting time for every device.

In [ ]:
query_upload_last = "SELECT LAST(UPLOAD), time FROM SPEEDTEST_IPERF_UPLOAD WHERE PROVIDER!='iperf' AND time <= '"+starting_point+"' AND UPLOAD>0 GROUP BY SK_PI;"
result_upload_last=get_stats_influxdb(client_influx=client,
                               query_influx=query_upload_last,
                               stat_name='time',
                               device_numbers=device_numbers)

In [ ]:
query_upload_first = "SELECT FIRST(UPLOAD), time FROM SPEEDTEST_IPERF_UPLOAD WHERE PROVIDER!='iperf' AND time <= '"+starting_point+"' AND UPLOAD>0 GROUP BY SK_PI;"
result_upload_first=get_stats_influxdb(client_influx=client,
                               query_influx=query_upload_first,
                               stat_name='time',
                               device_numbers=device_numbers)

In [ ]:
print("Speedtest reporting times:")
data=[]
for i in range(len(device_numbers)):
    try:
        result_upload_first[i] = dateutil.parser.parse(result_upload_first[i]).strftime('%Y-%m-%d %H:%M:%S')
    except:
        result_upload_first[i]=None
    try:    
        result_upload_last[i] = dateutil.parser.parse(result_upload_last[i]).strftime('%Y-%m-%d %H:%M:%S')
    except:
        result_upload_last[i]=None
    print("Device: ", device_numbers[i],"  was reporting from ", result_upload_first[i], " to ",result_upload_last[i])
    trace = go.Scatter(x=[result_upload_first[i],result_upload_last[i]],y=[device_numbers[i],device_numbers[i]], 
                       name = device_numbers[i],marker=dict(color=colors[i]))
    data.append(trace)
layout = dict(title = "Device reporting times(speedtest)",xaxis=dict(title="Time"),
        yaxis=dict(title="Device Number"))
fig = go.Figure(data=data, layout=layout)
iplot(fig)

### Statistics by device

In [ ]:
query_download = "SELECT * FROM SPEEDTEST_IPERF_DOWNLOAD WHERE PROVIDER!='iperf'AND DOWNLOAD>0 AND time >='"+starting_point+"'-"+\
                         time_interval;
download_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_download,
                                          table_name='SPEEDTEST_IPERF_DOWNLOAD')

In [ ]:
query_upload = "SELECT * FROM SPEEDTEST_IPERF_UPLOAD WHERE PROVIDER!='iperf'AND UPLOAD>0 AND time >= '"+starting_point+"'-"+\
                         time_interval;
upload_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_upload,
                                          table_name='SPEEDTEST_IPERF_UPLOAD')

In [ ]:
query_ping = "SELECT * FROM SPEEDTEST_IPERF_PING WHERE PROVIDER!='iperf'AND PING>0 AND time >= '"+starting_point+"'-"+\
                         time_interval;
ping_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_ping,
                                          table_name='SPEEDTEST_IPERF_PING')
ping_df = ping_df[ping_df.PING != 1800000.000] ## Removed outier that was found in the ms sql table

In [ ]:
download_summary=mean_max_median_by1(download_df,'DOWNLOAD')
device_numbers_d=download_summary["SK_PI"].unique()
download_line=go.Scatter(x=device_numbers_d,y=[50] * len(device_numbers_d), mode='markers',marker=dict(color='red'), name='50Mps')
combined_bar_plot_3traces(xvalues=download_summary["SK_PI"],
                         yvalues1=download_summary["max"],
                         yvalues2=download_summary["mean"],
                         yvalues3=download_summary["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Download speed over the last "+time_interval+ " starting from "+ starting_point,
                         ytitle="Mbps",
                         line=download_line,
                         stack=False)

In [ ]:
upload_summary=mean_max_median_by1(upload_df,'UPLOAD')
device_numbers_u=upload_summary["SK_PI"].unique()
upload_line=go.Scatter(x=device_numbers_u,y=[10] * len(device_numbers_u), mode='markers',marker=dict(color='red'), name='10Mbps')

combined_bar_plot_3traces(xvalues=upload_summary["SK_PI"],
                         yvalues1=upload_summary["max"],
                         yvalues2=upload_summary["mean"],
                         yvalues3=upload_summary["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Upload speed over the last "+time_interval+ " starting from "+ starting_point,
                         ytitle="Mbps",
                         line=upload_line,
                         stack=False)

In [ ]:
ping_summary=mean_max_median_by1(ping_df,'PING')
combined_bar_plot_3traces(xvalues=ping_summary["SK_PI"],
                         yvalues1=ping_summary["max"],
                         yvalues2=ping_summary["mean"],
                         yvalues3=ping_summary["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Ping latency over the last "+time_interval+ " starting from "+ starting_point,
                         ytitle="Miliseconds",
                         stack=False)

In [ ]:
simple_boxplot(dataframe=download_df,plot_value='DOWNLOAD',sort_value='SK_PI',
               title="Download speed over the last "+time_interval+ " starting from "+ starting_point, 
               ytitle="Mbps")

In [ ]:
simple_boxplot(dataframe=upload_df,plot_value='UPLOAD',sort_value='SK_PI',
               title="Upload speed over the last "+time_interval+ " starting from "+ starting_point, 
               ytitle="Mbps")

In [ ]:
simple_boxplot(dataframe=ping_df,plot_value='PING',sort_value='SK_PI',
               title="Ping latency over the last "+time_interval+ " starting from "+ starting_point, 
               ytitle="Miliseconds")

### Test servers
Are there any NaNs or empty strings?

In [ ]:
print(sum(pd.isnull(upload_df["TEST_SERVER"])))
print(sum(pd.isnull(upload_df["PROVINCE"])))
print(sum(upload_df["TEST_SERVER"] == ''))
print(sum(upload_df["PROVINCE"] == ''))

In [ ]:
print("There are ",len(upload_df["PROVINCE"].unique()), " provinces: ", upload_df["PROVINCE"].unique(), "\n")
print("There are ",len(upload_df["TEST_SERVER"].unique()), " test servers", upload_df["TEST_SERVER"].unique(), "\n")

pd.options.mode.chained_assignment = None
upload_df["server"] = upload_df["TEST_SERVER"] + " " +upload_df["PROVINCE"]
print("There are ",len(upload_df["server"].unique()), " test servers+provinces", upload_df["server"].unique(), "\n")

#### How many test servers per device?

In [ ]:
pd.options.mode.chained_assignment = None
upload_df["server"] = upload_df["TEST_SERVER"] + " " +upload_df["PROVINCE"]
download_df["server"] = download_df["TEST_SERVER"] + " " +download_df["PROVINCE"]
ping_df["server"] = ping_df["TEST_SERVER"] + " " +ping_df["PROVINCE"]

points_by_device=upload_df.groupby(['SK_PI']).size().reset_index(name='counts').sort_values('SK_PI')

test_servers_per_device = upload_df.groupby('SK_PI').apply(lambda x: x['server'].unique()).apply(pd.Series)
test_servers_per_device['server_count'] = test_servers_per_device.apply(lambda x: x.count(), axis=1)
test_server_counts_per_device = pd.Series(test_servers_per_device['server_count']).value_counts().reset_index()
points_by_device_by_server=upload_df.groupby(['SK_PI', 'server']).size().reset_index()

In [ ]:
simple_bar_plot(xvalues=test_servers_per_device.index,
                yvalues=test_servers_per_device["server_count"],
                name = "Number of providers",title="Number of test servers per device over the last "+time_interval+ " starting from "+ starting_point,
                ytitle="Number of providers")

In [ ]:
simple_pie_chart(labels=test_server_counts_per_device['index'],
            values=test_server_counts_per_device['server_count'],
                 title="Number of test servers per device over the last "+time_interval+ " starting from "+ starting_point,)

In [ ]:
combined_bar_plot_multitraces(dataframe=points_by_device_by_server,
                              device_numbers=device_numbers,
                              points_by_device=points_by_device,
                              sort_value='server',
                              ytitle="Percentage of data points",
                              title="Percentage of data points per device per test server over the last "+time_interval+ " starting from "+ starting_point)

In [ ]:
upload_summary=mean_max_median_by1(upload_df,'UPLOAD',index_col='server')
upload_line=go.Scatter(x=upload_summary["server"].unique(),y=[10] * len(upload_summary["server"].unique()), mode='markers',marker=dict(color='red'), name='10Mps')
combined_bar_plot_3traces(xvalues=upload_summary["server"],
                         yvalues1=upload_summary["max"],
                         yvalues2=upload_summary["mean"],
                         yvalues3=upload_summary["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Upload speed by test server over the last "+time_interval+ " starting from "+ starting_point,
                         ytitle="Mbps",
                         xtitle="Provider",
                         line=upload_line,
                         stack=False,
                         margin=True)

In [ ]:
download_summary=mean_max_median_by1(download_df,'DOWNLOAD',index_col='server')
download_line=go.Scatter(x=upload_summary['server'].unique(),y=[50] * len(upload_summary['server'].unique()), mode='markers',marker=dict(color='red'), name='50Mps')
combined_bar_plot_3traces(xvalues=download_summary['server'],
                         yvalues1=download_summary["max"],
                         yvalues2=download_summary["mean"],
                         yvalues3=download_summary["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Download speed by test server over the last "+time_interval+ " starting from "+ starting_point,
                         ytitle="Mbps",
                         xtitle="Test server",
                         line=download_line,
                         stack=False,
                         margin=True)

In [ ]:
ping_summary=mean_max_median_by1(ping_df,'PING',index_col='server')
combined_bar_plot_3traces(xvalues=ping_summary['server'],
                         yvalues1=ping_summary["max"],
                         yvalues2=ping_summary["mean"],
                         yvalues3=ping_summary["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Ping latency by  test server over the last "+time_interval+ " starting from "+ starting_point,
                         ytitle="Miliseconds",
                         xtitle="Test server",
                         stack=False,
                         margin=True)

### Providers
Are there any NaNs or empty strings?

In [ ]:
print(sum(pd.isnull(upload_df["PROVIDER"])))
print(sum(upload_df["PROVIDER"] == ''))

In [ ]:
print("There are ",len(upload_df["PROVIDER"].unique()), " providers", upload_df["PROVIDER"].unique(), "\n")

#### How many providers per device?

In [ ]:
points_by_device=upload_df.groupby(['SK_PI']).size().reset_index(name='counts').sort_values('SK_PI')
providers_per_device = upload_df.groupby('SK_PI').apply(lambda x: x["PROVIDER"].unique()).apply(pd.Series)
providers_per_device['provider_count'] = providers_per_device.apply(lambda x: x.count(), axis=1)
provider_counts_per_device = pd.Series(providers_per_device['provider_count']).value_counts().reset_index()
points_by_device_by_provider=upload_df.groupby(['SK_PI', 'PROVIDER']).size().reset_index()

In [ ]:
simple_bar_plot(xvalues=providers_per_device.index,
                yvalues=providers_per_device["provider_count"],
                name = "Number of providers",title="Number of providers per device over the last "+time_interval + " starting from "+ starting_point,
                ytitle="Number of providers")

In [ ]:
simple_pie_chart(labels=provider_counts_per_device['index'],
            values=provider_counts_per_device["provider_count"],title="Number of providers per device over the last "+time_interval+ " starting from "+ starting_point)

In [ ]:
combined_bar_plot_multitraces(dataframe=points_by_device_by_provider,
                              device_numbers=device_numbers,
                              points_by_device=points_by_device,
                              sort_value='PROVIDER',
                              ytitle="Percentage of data",
                              title="Percentage of data points per device per service provider over the last "+time_interval+ " starting from "+ starting_point,)

In [ ]:
upload_summary=mean_max_median_by1(upload_df,'UPLOAD',index_col='PROVIDER')
upload_line=go.Scatter(x=upload_summary["PROVIDER"].unique(),y=[10] * len(upload_summary["PROVIDER"].unique()), mode='markers',marker=dict(color='red'), name='10Mbps')
combined_bar_plot_3traces(xvalues=upload_summary["PROVIDER"],
                         yvalues1=upload_summary["max"],
                         yvalues2=upload_summary["mean"],
                         yvalues3=upload_summary["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Upload speed by provider over the last "+time_interval+ " starting from "+ starting_point,
                         ytitle="Mbps",
                         xtitle="Provider",
                         line=upload_line,
                         stack=False,
                         margin=True)

In [ ]:
download_summary=mean_max_median_by1(download_df,'DOWNLOAD',index_col='PROVIDER')
download_line=go.Scatter(x=upload_summary["PROVIDER"].unique(),y=[50] * len(upload_summary["PROVIDER"].unique()), mode='markers',marker=dict(color='red'), name='50Mbps')
combined_bar_plot_3traces(xvalues=download_summary["PROVIDER"],
                         yvalues1=download_summary["max"],
                         yvalues2=download_summary["mean"],
                         yvalues3=download_summary["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Download speed by provider over the last "+time_interval+ " starting from "+ starting_point,
                         ytitle="Mbps",
                         xtitle="Provider",
                         line=download_line,
                         stack=False,
                         margin=True)

In [ ]:
ping_summary=mean_max_median_by1(ping_df,'PING',index_col='PROVIDER')
combined_bar_plot_3traces(xvalues=ping_summary["PROVIDER"],
                         yvalues1=ping_summary["max"],
                         yvalues2=ping_summary["mean"],
                         yvalues3=ping_summary["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Ping latency by provider over the last "+time_interval+ " starting from "+ starting_point,
                         ytitle="Miliseconds",
                         xtitle="Provider",
                         stack=False,
                         margin=True)

Bell MTS has largest maximum spikes and TerraGo has largest median/average. 

### How often data is collected?

Let's ceck device number 5 with the larges number of datapoints and (8 test servers, 2 providers)  and device number 8 with largest number of providers (3) and 4 test servers?

In [ ]:
upload_df

In [ ]:
upload_df.loc[upload_df['SK_PI']==11][['time','SK_PI', 'PROVIDER', 'server','UPLOAD']].head(10)

In [ ]:
upload_df.loc[upload_df['SK_PI']==11][['time','SK_PI', 'PROVIDER', 'server','UPLOAD']].tail(5)

In [ ]:
upload_df.loc[upload_df['SK_PI']==34][['time','SK_PI', 'PROVIDER', 'server','UPLOAD']].head(5)

In [ ]:
upload_df.loc[upload_df['SK_PI']==34][['time','SK_PI', 'PROVIDER', 'server','UPLOAD']].tail(5)

 Looks like data is collected with 3 hours 42 mins intervals using different test servers. 
 How is it determined which test server to use?